In [3]:
!pip install catboost

# Импорты

In [4]:
import numpy as np # для работы с массивами
import pandas as pd # для работы с DataFrame 

# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split


# инструменты для оценки точности модели 
from sklearn import metrics 
from sklearn.metrics import mean_absolute_percentage_error as mape


# модуль для работы с полиноминальными признаками
from sklearn.preprocessing import PolynomialFeatures


# Модули для нахождения выбросов
from sklearn.cluster import DBSCAN

# Модули для маштабирования
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler


# Модуль для вывода прогресса
from tqdm.notebook import tqdm

# Модули алгоритмов
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor  
import xgboost as xgb
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

# модуль для подбора параметров модели
from sklearn.model_selection import GridSearchCV

# keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.activations import *
from tensorflow.keras.applications import *

#import keras
from keras import optimizers
import albumentations

# Модули для визуализации
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# Настройки Pandas

In [5]:
# Настройки Pandas
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', 20)
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', 20)

# Функции

In [6]:
def load(url,col_del):
    df = pd.read_csv(url).drop(col_del, axis = 1)
    return df


def df_column_uniquify(df):
    df_columns = df.columns
    new_columns = []
    for item in df_columns:
        counter = 0
        newitem = item
        while newitem in new_columns:
            counter += 1
            newitem = "{}_{}".format(item, counter)
        new_columns.append(newitem)
    df.columns = new_columns
    return df


def PreProc(data, log_col,colum):
    # Отделяем таргет
    y = data.target
    
    # Удаляем таргет из данных
    data.drop(['target'], inplace = True,axis = 1)
    
    # Логарифмируем выбраные столбцы
    data[log_col] = np.log(data[log_col])
    
    # Закодируем категориальные признаки.
    for colum in cat:
        data[colum] = data[colum].astype('category').cat.codes

    # Нормализируем данные (RobustScaler должен быть устойчив к выбросам).
    scaler = RobustScaler() 
    for column in num_col:
        data[column] = scaler.fit_transform(data[[column]])[:,0]
    
    
    return y, data

In [7]:
url = '../input/df-after-fe-eda-1/df_after_FE_EDA (1).csv'

col_del = ['Unnamed: 0','fireplace','cooling','name','data_distance','data_grades',
           'rating','rating_sum','rating_mean','rating_median','data_distance_sum','data_distance_mean','data_distance_median']

In [8]:
# Загружаем данные
df = load(url,col_del)
df['mean_state_sqft'] = df['mean_state_sqft'] = df['state'].map(df.groupby('state')['sqft'].mean()) #!

In [9]:
# При загрузке датасета проявляються столбцы с одинаковыми названиями применим функцию которая это исправит.
df = df_column_uniquify(df)

In [10]:
# Делим данные на трейн и вал.тест
Train, val, = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

In [11]:
data = Train
#data = df.copy()



# CatBoost

In [12]:
# Разделим признаки на числовые и категориальные
num_col = ['baths','sqft','beds','stories','PrivatePool','year_built',
           'remodeled_year','lotsize','price_sqft','mean_city_data_distance','year_built_mean_city',
           'year_remodeled_mean_city','mean_state_rating','mean_state_data_distance',
           'year_built_mean_state','year_remodeled_mean_state','beds__mean_state','median_city_sqft','mean_city_sqft','max_city_sqft','min_city_sqft',
           'min_state_sqft','mean_state_sqft','median_city_stories','mean_city_stories','min_state_stories',
           'mean_city_PrivatePool'
          ]


cat = ['status', 'propertyType', 'street','city', 'sqft', 
       'zipcode','state', 'heating','parking','propertyType_1','parking_',]


log_col = ['lotsize']

In [13]:
# Предобрабатываем данные трейна
y, data = PreProc(data, log_col,cat)

In [14]:
# Предобрабатываем данные вал теста
y_val, data_val = PreProc(val, log_col,cat)

In [15]:
# split данных
X_train, X_test, y_train, y_test = train_test_split(data, np.log(y), test_size=0.2, shuffle=True, random_state=42)

In [16]:
# Обучаем
model_cat = CatBoostRegressor(iterations = 40000,
                          learning_rate = 0.01,
                          l2_leaf_reg=2,
                          random_seed = 42,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                          task_type='GPU',

                         )
model_cat.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=1000,
         use_best_model=True,
         #plot=True
         )

In [17]:
# Результат метрики на трейне В 2 раза лучше чем наивная модель
catboost_predict_train = model_cat.predict(X_test)
print(f"TEST mape: {(mape(np.exp(y_test), np.exp(catboost_predict_train)))*100:0.2f}%")

In [18]:
# Результат метрики на Вал тесте
catboost_predict_val = model_cat.predict(data_val)
print(f"TEST mape: {(mape(y_val, np.exp(catboost_predict_val)))*100:0.2f}%")

In [19]:
catboost_predict_train = pd.Series(catboost_predict_train)
catboost_predict_train.to_csv('./catboost_predict_train.csv')

In [20]:
catboost_predict_val = pd.Series(catboost_predict_val)
catboost_predict_val.to_csv('./catboost_predict_val.csv')

In [21]:
# Смотрим какие признаки были значимы для модели
plt.rcParams['figure.figsize'] = (13,10)
feat_importances = pd.Series(model_cat.feature_importances_, index=X_test.columns)
feat_importances.nlargest(50).plot(kind='barh')

# Подготовка данных для модели Random Forest


In [22]:
# Делим данные на трейн и вал.тест
Train, val, = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

In [23]:
data = Train
# Загружаем данные
#data = df.copy()

In [24]:
# Разделим признаки на числовые и категориальные
num_col = ['baths','sqft','beds','stories','PrivatePool','year_built',
           'remodeled_year','lotsize','price_sqft','mean_city_data_distance','year_built_mean_city',
           'year_remodeled_mean_city','mean_state_rating','mean_state_data_distance',
           'year_built_mean_state','year_remodeled_mean_state','beds__mean_state','median_city_sqft','mean_city_sqft','max_city_sqft','min_city_sqft',
           'min_state_sqft','mean_state_sqft','median_city_stories','mean_city_stories','min_state_stories',
           'mean_city_PrivatePool'
          ]


cat = ['status', 'propertyType', 'street','city', 'sqft', 
       'zipcode','state', 'heating','parking','propertyType_1','parking_',]


log_col = ['lotsize']

In [25]:
# При загрузке датасета проявляються столбцы с одинаковыми названиями применим функцию которая это исправит.
data = df_column_uniquify(data)

In [26]:
# Предобрабатываем данные трейна
y, data = PreProc(data, log_col,cat)

In [27]:
# Предобрабатываем данные вал теста
y_val, data_val = PreProc(val, log_col,cat)

In [28]:
# Заполяем оставшиеся пропуски средним
def fillna_mean(df):
    df = df.fillna(df.mean())
    return df

# Заполяем оставшиеся пропуски средним на трейне
data = fillna_mean(data)

# Заполяем оставшиеся пропуски средним на вал.тесте
data_val = fillna_mean(data_val)

In [29]:
# split данных
X_train, X_test, y_train, y_test = train_test_split(data, np.log(y), test_size=0.2, shuffle=True, random_state=42)

# Random Forest



### Grid for forest

In [30]:
#estimator = RandomForestRegressor()
#param_grid = { 
#            "n_estimators"      : [10,50,100],
#            "max_features"      : ["sqrt", "log2"],
#            "min_samples_split" : [2,4,8],
#            "bootstrap": [True],
#            }
#grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5, verbose=1)
#
#grid.fit(X_train, y_train)

In [31]:
#print(grid.best_score_) 
#print(grid.best_params_)

In [32]:
# Обучаем лес
forest = RandomForestRegressor(random_state=42, n_estimators=100,verbose=1, n_jobs=-1)

forest.fit(X_train, y_train)

In [33]:
# Результат метрики на трейне
forest_predict_train = forest.predict(X_test)
print(f"TEST mape: {(mape(np.exp(y_test), np.exp(forest_predict_train)))*100:0.2f}%")

In [34]:
# Результат метрики на вал.тесте
forest_predict_val = forest.predict(data_val)
print(f"TEST mape: {(mape(y_val, np.exp(forest_predict_val)))*100:0.2f}%")

In [35]:
forest_predict_train = pd.Series(forest_predict_train)
forest_predict_train.to_csv('./forest_predict_train.csv')

In [36]:
forest_predict_val = pd.Series(forest_predict_val)
forest_predict_val.to_csv('./forest_predict_val.csv')

In [37]:
# Смотрим какие признаки были значимы для модели
plt.rcParams['figure.figsize'] = (13,10)
feat_importances = pd.Series(forest.feature_importances_, index=X_test.columns)
feat_importances.nlargest(50).plot(kind='barh')

# GradientBoostingRegressor

In [38]:
# Делим данные на трейн и вал.тест
Train, val, = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

In [39]:
data = Train
# Загружаем данные
#data = df.copy()

In [40]:
# Разделим признаки на числовые и категориальные
num_col = ['baths','sqft','beds','stories','PrivatePool','year_built',
           'remodeled_year','lotsize','price_sqft','mean_city_data_distance','year_built_mean_city',
           'year_remodeled_mean_city','mean_state_rating','mean_state_data_distance',
           'year_built_mean_state','year_remodeled_mean_state','beds__mean_state','median_city_sqft','mean_city_sqft','max_city_sqft','min_city_sqft',
           'min_state_sqft','mean_state_sqft','median_city_stories','mean_city_stories','min_state_stories',
           'mean_city_PrivatePool'
          ]


cat = ['status', 'propertyType', 'street','city', 'sqft', 
       'zipcode','state', 'heating','parking','propertyType_1','parking_',]


log_col = ['lotsize']

In [41]:
# При загрузке датасета проявляються столбцы с одинаковыми названиями применим функцию которая это исправит.
data = df_column_uniquify(data)

In [42]:
# Предобрабатываем данные трейна
y, data = PreProc(data, log_col,cat)

In [43]:
# Предобрабатываем данные вал теста
y_val, data_val = PreProc(val, log_col,cat)

In [44]:
# Заполяем оставшиеся пропуски средним на трейне
data = fillna_mean(data)

# Заполяем оставшиеся пропуски средним на вал.тесте
data_val = fillna_mean(data_val)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(data, np.log(y), test_size=0.2, shuffle=True, random_state=42)

In [46]:
#estimator = GradientBoostingRegressor()
#param_grid = { 
#            "n_estimators"      : [10,100,500],
#            "max_features"      : ["sqrt", "log2"],
#            "min_samples_split" : [2,4,8],
#            "bootstrap": [True],
#            }
#grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5, verbose=1)

#grid.fit(X_train, y_train)


In [47]:
grad = GradientBoostingRegressor(n_estimators=200,learning_rate=0.1,random_state=42, verbose = 1, max_depth = 10)
grad.fit(X_train, y_train)
grad_test_predict = grad.predict(X_test)

In [48]:
# Результат метрики на трейне
grad_test_predict_train = grad.predict(X_test)
print(f"TEST mape: {(mape(np.exp(y_test), np.exp(grad_test_predict_train)))*100:0.2f}%")

In [49]:
# Результат метрики на вал.тесте
grad_test_predict_val = grad.predict(data_val)
print(f"TEST mape: {(mape(y_val, np.exp(grad_test_predict_val)))*100:0.2f}%")

In [50]:
# Смотрим какие признаки были значимы для модели
plt.rcParams['figure.figsize'] = (13,10)
feat_importances = pd.Series(grad.feature_importances_, index=X_test.columns)
feat_importances.nlargest(50).plot(kind='barh')

In [51]:
grad_test_predict_train = pd.Series(grad_test_predict_train)
grad_test_predict_train.to_csv('./grad_test_predict_train.csv')

In [52]:
grad_test_predict_val = pd.Series(grad_test_predict_val)
grad_test_predict_val.to_csv('./grad_test_predict_val.csv')

# Blending

In [ ]:
catboost_predict_train = pd.read_csv('./catboost_predict_train.csv').drop(['Unnamed: 0'], axis = 1)
forest_predict_train = pd.read_csv('./forest_predict_train.csv').drop(['Unnamed: 0'], axis = 1)
grad_test_predict_train = pd.read_csv('./grad_test_predict_train.csv').drop(['Unnamed: 0'], axis = 1)

In [53]:
blend_predict_train = (np.exp(catboost_predict_train.values) + np.exp(forest_predict_train.values) + np.exp(grad_test_predict_train.values)) / 3
print(f"TEST mape: {(mape(np.exp(y_test), blend_predict_train))*100:0.2f}%")

In [ ]:
catboost_predict_val = pd.read_csv('./catboost_predict_val.csv').drop(['Unnamed: 0'], axis = 1)
forest_predict_val = pd.read_csv('./forest_predict_val.csv').drop(['Unnamed: 0'], axis = 1)
grad_test_predict_val = pd.read_csv('./grad_test_predict_val.csv').drop(['Unnamed: 0'], axis = 1)

In [55]:
blend_predict_val = (np.exp(catboost_predict_val.values) + np.exp(forest_predict_val.values) + np.exp(grad_test_predict_val.values)) / 3
print(f"TEST mape: {(mape(y_val, blend_predict_val))*100:0.2f}%")

# Стекинг

In [ ]:
# По причине долгих вычислений на kaggle, пришлось ограничить число деревьев в каждом методе до 100.
estimators = [
              ('Cat', CatBoostRegressor(iterations=2000,
                                        learning_rate = 0.01,
                                        random_seed=42,
                                        silent=True,
                                        l2_leaf_reg=2,
                                        task_type='GPU')),
    
              ('Grad', GradientBoostingRegressor(min_samples_split=2,
                               learning_rate=0.1,
                               n_estimators=100,
                               random_state = 42,
                               max_depth = 10 )),
    
              ('forest', RandomForestRegressor(n_jobs=-1,max_features= 'auto' ,
                                               n_estimators=100,
                                               oob_score = True, random_state=42))
               
               ]

st_ensemble = StackingRegressor(estimators=estimators, final_estimator=LinearRegression(),verbose = 1)

# оцениваем точность
st_ensemble.fit(X_train, np.log(y_train))




In [ ]:
stack = np.exp(st_ensemble.predict(X_test))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(stack)))*100:0.10f}%")

# **Выводы:** 
1. Разделили данные на числовые и категориальные признаки.
2. Провели анализ числовых и категориальных признаков.
3. Очистили данные  от выбросов (там где это принесло результат).
4. Создали новые признаки и отсеяли плохие из них в процесе экспериментов.
5. Провели анализ распределения и кореляции признаков.  
6. Из категориальных признаков выделили dummy признаки.
7. Выбрали метрику МАРЕ так как с ней удобно сравнивать модели.
8. Было опробовано несколько моделей.
*   GradientBoostingRegressor 10.35%  вал.тест - 13.22%
*   RandomForestRegressor трейн - 9.49%, вал.тест - 12.93%
*   StackingRegressor. 
*   CatBoostRegressor трейн - 11.39%, вал.тест - 10.79%
9. Применили Blending трейн - 9.93%, вал.тест - 11.67%
* Лучший результат показал на трейне показал RandomForestRegressor.
* Лучший результат показал на вал.тесте показал CatBoostRegressor.
* Разница значений на трейнах и вал.тестах около 3%, между блендингами 1.6%.
10. Что не удалось: 

*   Не удалось применить DL из-за того, что датасет слишком разросся и не вле6зает в RAM как googlecolab так и Kaggle. Нужно подумвть,что с этим делать. Так как нейронная сеть работает иначе чем ансамбли блендинг с ее участием может показать хороший результат.  
*   Пока не получилось применить GridSearch для подбора гиперпараметров моделей из-за продолжительности процеса. 

11. Что еще можно зделать:
*   Возможно еще привлечь данные о населении городов, отдаленности от обласных центров(столиц штатов), рейтинг криминальностигородов(раенов), доступность центров развлечений(магазинов) и т.д.
12. Датасет был слишком загрязнен. И в процесе очистки почти стопроцентно были допущены ошибки. Так как garbage in, garbage out, это очевидно уменьшает score.Это можно решить более глубокой обработкой текста(в реальной жизни скорее всего плохой вариант). Или (представим, что мы владельцы ресурса где данные собирались) зделать процес записи данных в виде бланка с указаными возможными категориями. 

# 